In [1]:
import csv
from bs4 import BeautifulSoup
import requests
import pandas as pd
!pip install requests PyPDF2

import os
import requests
from PyPDF2 import PdfReader


# Fetch the webpage
url = "https://aclanthology.org/events/acl-2023/#2023acl-long"
response = requests.get(url)

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <div> elements with id "2023acl-long"
div_2023acl_long = soup.find_all('div', id='2023acl-long')

# Initialize lists to store names and abstracts
names = []

# Extract the data
if div_2023acl_long:
    for div in div_2023acl_long:
        # Find all <p> elements with class "d-sm-flex align-items-stretch"
        p_tags = div.find_all('p', class_='d-sm-flex align-items-stretch')
        for p_tag in p_tags:
            # Find all <span> elements with class "d-block"
            span_tags = p_tag.find_all('span', class_='d-block')
            for span_tag in span_tags:
                # Find all <strong> elements
                strong_tags = span_tag.find_all('strong')
                for strong_tag in strong_tags:
                    # Find all <a> elements with class "align-middle"
                    a_tags = strong_tag.find_all('a', class_='align-middle')
                    for a_tag in a_tags:
                        # Extract the name from the <a> tag
                        name = a_tag.get_text(strip=True)
                        names.append(name)
                        #print(name)
                        #print("\n")
    
else:
    print("Div with id '2023acl-long' not found on the page.")
    


# Create a dictionary from the lists
data = {'Name': names}

# Create a DataFrame
df = pd.DataFrame(data)  


# Base URL pattern for the papers
base_url = "https://aclanthology.org/2023.acl-long.{}.pdf"

# Initialize an empty list to store the text of each paper
papers_text = []


"""
let us say we want papers from 1 to 6 then a = 1, b = 6
"""

a = 1
b = 41



# Function to download PDF from URL and extract text
def extract_text_from_pdf(url):
    # Download PDF file from URL
    response = requests.get(url)
    with open("temp_pdf.pdf", "wb") as f:
        f.write(response.content)

    # Open downloaded PDF file and extract text
    with open("temp_pdf.pdf", "rb") as f:
        pdf_reader = PdfReader(f)
        num_pages = len(pdf_reader.pages)
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    index_conclusion = text.find("Conclusion")
    index_conclusions = text.find("Conclusions")
    index_references = text.find("References")
    
    # Determine the actual index of the conclusion
    if index_conclusions != -1:
        # Use "Conclusions" if found, otherwise fall back to "Conclusion"
        actual_index_conclusion = index_conclusions
    else:
        actual_index_conclusion = index_conclusion

    # Check if "References" is after "Conclusion" or "Conclusions"
    if index_references != -1 and actual_index_conclusion != -1 and index_references > actual_index_conclusion:
        # Trim the text before "References"
        text = text[:index_references]

    # If "References" is before "Conclusion", do not trim
    # You might want to handle this case separately depending on your needs
    else:
        # Handle the case where "References" appears before "Conclusion"
        pass 

    index_references = text.find("Abstract")
    text = text[index_references:] 
    
    
    # Delete temporary PDF file
    os.remove("temp_pdf.pdf")

    return text


# Loop through the range of papers (from 1 to 990)
for i in range(a, b):  # Change range to (1, 911) for all papers
    url = base_url.format(i)
    print(url)
    paper_text = extract_text_from_pdf(url)
    papers_text.append(paper_text)


# Create a dictionary from the lists
data = {'Name': names[a+1:b+1], 'TextData': papers_text[:b] }

# Create a DataFrame
df = pd.DataFrame(data)


# Define a list of new column names
#new_columns = ['Knowledge Extraction']

# Reindex the DataFrame with new columns (creates NaN values)
#df = df.reindex(columns=df.columns.tolist() + new_columns)

#df

DEPRECATION: bert-score 0.3.10 has a non-standard dependency specifier transformers>=3.0.0numpy. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of bert-score or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


https://aclanthology.org/2023.acl-long.1.pdf
https://aclanthology.org/2023.acl-long.2.pdf
https://aclanthology.org/2023.acl-long.3.pdf
https://aclanthology.org/2023.acl-long.4.pdf
https://aclanthology.org/2023.acl-long.5.pdf
https://aclanthology.org/2023.acl-long.6.pdf
https://aclanthology.org/2023.acl-long.7.pdf
https://aclanthology.org/2023.acl-long.8.pdf
https://aclanthology.org/2023.acl-long.9.pdf
https://aclanthology.org/2023.acl-long.10.pdf
https://aclanthology.org/2023.acl-long.11.pdf
https://aclanthology.org/2023.acl-long.12.pdf
https://aclanthology.org/2023.acl-long.13.pdf
https://aclanthology.org/2023.acl-long.14.pdf
https://aclanthology.org/2023.acl-long.15.pdf
https://aclanthology.org/2023.acl-long.16.pdf
https://aclanthology.org/2023.acl-long.17.pdf
https://aclanthology.org/2023.acl-long.18.pdf
https://aclanthology.org/2023.acl-long.19.pdf
https://aclanthology.org/2023.acl-long.20.pdf
https://aclanthology.org/2023.acl-long.21.pdf
https://aclanthology.org/2023.acl-long.22.p

In [2]:
import pandas as pd
import re

def clean_and_join_text(text):
    # Join all lines into a single paragraph
    joined_text = ' '.join(text.split('\n'))
    
    # Replace consecutive spaces with a single space
    cleaned_text = ' '.join(joined_text.split())
    
    # Correct hyphenated words that were split due to line breaks
    cleaned_text = re.sub(r'(\w+)\-\s*(\w+)', r'\1\2', cleaned_text)
    
    return cleaned_text

# Assuming df is your dataframe and 'TextData' is the column containing the text
df['TextData'] = df['TextData'].apply(clean_and_join_text)

# Display the cleaned and joined text
df


,Name,TextData
0,One Cannot Stand for Everyone! Leveraging Mult...,Abstract User simulators are agents designed t...
1,SafeConv: Explaining and Correcting Conversati...,Abstract One of the main challenges opendomain...
2,Detecting and Mitigating Hallucinations in Mac...,Abstract While the problem of hallucinations i...
3,Explainable Recommendation with Personalized R...,Abstract Explainable recommendation is a techn...
4,Binary and Ternary Natural Language Generation,Abstract Ternary and binary neural networks en...
5,Span-Selective Linear Attention Transformers f...,Abstract In schemaguided dialogue state tracki...
6,EMPre-training for Multi-party Dialogue Respon...,Abstract Dialogue response generation requires...
7,ACLM: A Selective-Denoising based Generative D...,Abstract Complex Named Entity Recognition (NER...
8,Natural Language to Code Generation in Interac...,"Abstract Computational notebooks, such as Jupy..."
9,Subset Retrieval Nearest Neighbor Machine Tran...,Abstract knearest-neighbor machine translation...


In [3]:
import pandas as pd
import re

# Sample dataset


# Headers to match
headers = ['Abstract', 'Background', 'Introduction', 'Method', 'Methodology', 'Approach', 
           'Analysis', 'Experiment Results', 'Experiments', 'Experimental Setup', 
           'Evaluation', 'Related Work','Related Works', 'Conclusion','Conclusions']

# Create columns for each header
for header in headers:
    df[header] = ''

def extract_text(row):
    text = row['TextData']
    
    for header in headers:
        # Try to find the header in the text
        match = re.search(fr'\b{header}\b', text, flags=re.IGNORECASE)
        if match:
            start_index = match.start()
            # Find the next header or end of text
            next_header_index = len(text)
            for next_header in headers:
                next_match = re.search(fr'\b{next_header}\b', text[start_index:], flags=re.IGNORECASE)
                if next_match:
                    next_header_index = start_index + next_match.start()
                    break
            
            # Extract the text corresponding to the current header
            if next_header_index > start_index:
                header_text = text[start_index:next_header_index]
            else:
                header_text = text[start_index:]
            
            # Assign the extracted text to the corresponding column
            row[header] = header_text.strip()
    
    return row

# Apply the function to each row
df = df.apply(extract_text, axis=1)

# Drop the original 'TextData' column
df.drop('TextData', axis=1, inplace=True)

# Display the resulting DataFrame
df


,Name,Abstract,Background,Introduction,Method,Methodology,Approach,Analysis,Experiment Results,Experiments,Experimental Setup,Evaluation,Related Work,Related Works,Conclusion,Conclusions
0,One Cannot Stand for Everyone! Leveraging Mult...,Abstract User simulators are agents designed t...,Background Dialogue system. Taskoriented dialo...,Introduction Taskoriented dialogue systems aim...,method called MUST adaptive that balances i) t...,,"approach (Schatzmann et al., 2007; Schatzmann ...",Analysis and Discussions Convergences of MUST ...,,"experiments, we observed that the dialogue sys...",Experimental Setup Available user simulators. ...,"evaluation. Moreover, our results show that ou...",,,"Conclusion In this paper, we propose a framewo...",
1,SafeConv: Explaining and Correcting Conversati...,Abstract One of the main challenges opendomain...,,Introduction Safety of artificial intelligence...,method is onetime checking and rewriting—direc...,,,Analysis There are cases that can not be detox...,,Experiments show that the detected unsafe beha...,,evaluation/testing from the translated data wi...,Related Work Dialogue Safety Datasets Datasets...,,"Conclusion In this paper, we study how to expl...",
2,Detecting and Mitigating Hallucinations in Mac...,Abstract While the problem of hallucinations i...,"Background and Setting In this section, we des...",Introduction Hallucinations in machine transla...,method that evaluates the percentage of the so...,"methodology In this section, we explain the se...","approach that relies on external models. Next,...","analysis, the model was trained on the remaini...",,experiments.1 1 Introduction Hallucinations in...,,"evaluation, it might be beneficial to use mode...",,,,Conclusions We start by asking how far we can ...
3,Explainable Recommendation with Personalized R...,Abstract Explainable recommendation is a techn...,,Introduction Recent years have witnessed a gro...,method has been employed to identify and selec...,Methodology 4.1 Overview of Model Here we pres...,approach facilitates the generation of suffici...,analysis of prediction and explanation tasks p...,,experiments on three datasets show that our mo...,,"Evaluation Metrics For rating prediction, in o...",Related Work 2.1 Explainable Recommendation wi...,,"Conclusion In this paper, we propose a novel m...",
4,Binary and Ternary Natural Language Generation,Abstract Ternary and binary neural networks en...,,Introduction Generative pretrained transformer...,method to natural language generation tasks an...,,approach the problem with a mix of statisticsb...,"analysis In language generation tasks, the err...",,"Experiments In this section, we evaluate the e...",,evaluation metrics. XSUM CNN/DailyMail,Related Work Quantization has long been studie...,,Conclusion We have demonstrated high accuracy ...,
5,Span-Selective Linear Attention Transformers f...,Abstract In schemaguided dialogue state tracki...,,Introduction Dialogue State Tracking (DST) ref...,,,approach significantly improves upon existing ...,analysis on JGA. We find that SPLAT base outpe...,,experiments on the SchemaGuided Dialogue (SGD)...,Experimental Setup We describe our experimenta...,"evaluation, implementation details, baselines ...",Related Work Extractive DST. Following the tra...,,"Conclusion In this work we introduced SPLAT, a...",
6,EMPre-training for Multi-party Dialogue Respon...,Abstract Dialogue response generation requires...,,Introduction Inspired by the tremendous succes...,method. The official implementation of this pa...,Methodology To design a model for multiparty d...,approach that iteratively performs the expecta...,Analysis In order to get more insights into th...,,experiments have justified the feasibility and...,"experimental setup, including hyperparameter s...",Evaluation Metrics Table 1 shows the results o...,,Related Works 2.1 Pretraining for Response Gen...,Conclusion Most multiparty dialogue corpora ar...,
7,ACLM: A Selective-Denoising based Gener

In [4]:

# Apply the function to create the new merged column
df['Methodology'] = df['Method'] + ' ' + df['Methodology'] +' '+ df['Approach'] 
df['Experiments'] = df['Experiment Results'] = df['Experiments'] + ' ' + df['Experimental Setup'] +' '+df['Evaluation']+' '+ df['Analysis']
df['Related Work'] = df['Related Work']+' '+ df['Related Works']
df['Conclusion'] = df['Conclusion']+' '+ df['Conclusions']

df.drop('Analysis', axis=1, inplace=True)
df.drop('Experimental Setup', axis=1, inplace=True)
df.drop('Experiment Results', axis=1, inplace=True)
df.drop('Evaluation', axis=1, inplace=True)
df.drop('Method', axis=1, inplace=True)
df.drop('Approach', axis=1, inplace=True)
df.drop('Conclusions', axis=1, inplace=True)
df.drop('Related Works', axis=1, inplace=True)

df

,Name,Abstract,Background,Introduction,Methodology,Experiments,Related Work,Conclusion
0,One Cannot Stand for Everyone! Leveraging Mult...,Abstract User simulators are agents designed t...,Background Dialogue system. Taskoriented dialo...,Introduction Taskoriented dialogue systems aim...,method called MUST adaptive that balances i) t...,"experiments, we observed that the dialogue sys...",,"Conclusion In this paper, we propose a framewo..."
1,SafeConv: Explaining and Correcting Conversati...,Abstract One of the main challenges opendomain...,,Introduction Safety of artificial intelligence...,method is onetime checking and rewriting—direc...,Experiments show that the detected unsafe beha...,Related Work Dialogue Safety Datasets Datasets...,"Conclusion In this paper, we study how to expl..."
2,Detecting and Mitigating Hallucinations in Mac...,Abstract While the problem of hallucinations i...,"Background and Setting In this section, we des...",Introduction Hallucinations in machine transla...,method that evaluates the percentage of the so...,experiments.1 1 Introduction Hallucinations in...,,Conclusions We start by asking how far we can...
3,Explainable Recommendation with Personalized R...,Abstract Explainable recommendation is a techn...,,Introduction Recent years have witnessed a gro...,method has been employed to identify and selec...,experiments on three datasets show that our mo...,Related Work 2.1 Explainable Recommendation wi...,"Conclusion In this paper, we propose a novel m..."
4,Binary and Ternary Natural Language Generation,Abstract Ternary and binary neural networks en...,,Introduction Generative pretrained transformer...,method to natural language generation tasks an...,"Experiments In this section, we evaluate the e...",Related Work Quantization has long been studie...,Conclusion We have demonstrated high accuracy ...
5,Span-Selective Linear Attention Transformers f...,Abstract In schemaguided dialogue state tracki...,,Introduction Dialogue State Tracking (DST) ref...,approach significantly improves upon existin...,experiments on the SchemaGuided Dialogue (SGD)...,Related Work Extractive DST. Following the tra...,"Conclusion In this work we introduced SPLAT, a..."
6,EMPre-training for Multi-party Dialogue Respon...,Abstract Dialogue response generation requires...,,Introduction Inspired by the tremendous succes...,method. The official implementation of this pa...,experiments have justified the feasibility and...,Related Works 2.1 Pretraining for Response Ge...,Conclusion Most multiparty dialogue corpora ar...
7,ACLM: A Selective-Denoising based Generative D...,Abstract Complex Named Entity Recognition (NER...,Background and Related Work Complex NER Backgr...,Introduction Named Entity Recognition (NER) is...,"method (Zhou and Chen, 2021) (previously evalu...",experiments reveal that the performance of the...,Related Work Complex NER,"Conclusion In this paper, we propose ACLM, a n..."
8,Natural Language to Code Generation in Interac...,"Abstract Computational notebooks, such as Jupy...",background material and hints in tutorial note...,Introduction Data science is the process of ex...,Method Length All / pandas JuICe (Agashe et al...,Experiments Models We evaluate PACHINC Oand st...,Related Work Automating Data Science The amoun...,"Conclusion In this paper we present ARCADE , a..."
9,Subset Retrieval Nearest Neighbor Machine Tran...,Abstract knearest-neighbor machine translation...,,Introduction Neural machine translation (NMT) ...,method improves the translation performance of...,"experiments on the WMT’19 Germanto-English, th...",Related Work The ﬁrst type of examplebased mac...,"Conclusion In this paper, we proposed “Subset ..."


The purpose of the authors: 'Abstract', 'Background', 'Introduction', 'Related Work'
The contribution of the paper: 'Introduction', 'Related Work', 'Methodology'
The method of the paper: 'Methodology'
The dataset used in the paper: 'Experiments'
The findings or conclusions of the paper: 'Conclusion'
The future work of the paper: 'Conclusion'

In [5]:
import pandas as pd

# Assuming df is your DataFrame containing the data you want to save

# Function to clean problematic characters
def clean_text(text):
    try:
        # Encode text to handle surrogate characters
        cleaned_text = text.encode('utf-8', errors='ignore').decode('utf-8')
        return cleaned_text
    except:
        return None

# Apply clean_text function to all columns of DataFrame
df_cleaned = df.applymap(clean_text)

# Export cleaned DataFrame to JSON
json_file_path = 'NewData_papers_upto_40.json'
df_cleaned.to_json(json_file_path, orient='records', force_ascii=False)
print(f"Data saved to {json_file_path}")


Data saved to NewData_papers_upto_40.json


C:\Users\harsh\AppData\Local\Temp\ipykernel_35632\454867521.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(clean_text)


In [6]:
# Specify the file path for saving the JSON file locally
json_file_path = r'C:\Users\harsh\Downloads\INFO_5731\NewData_papers_upto_40.json'

# Export cleaned DataFrame to JSON with specified file path
df_cleaned.to_json(json_file_path, orient='records', force_ascii=False)

print(f"Data saved to {json_file_path}")

Data saved to C:\Users\harsh\Downloads\INFO_5731\NewData_papers_upto_40.json


## NEW Data creation finished

In [38]:
import pandas as pd
import openai

# Set your OpenAI API key here
openai.api_key = 'sk-proj-nhSzo2tXsAnLAays3zfOT3BlbkFJFygVU07ChBJMJj6f08cd'

# Function to extract information using ChatGPT based on category
def extract_info_with_chatgpt(category, df):
    try:
        # Define mappings of category to corresponding columns
        category_mappings = {
            'purpose': ['Abstract', 'Background', 'Introduction', 'Related Work'],
            'contribution': ['Introduction', 'Related Work', 'Methodology'],
            'method': ['Methodology'],
            'dataset': ['Experiments'],
            'findings': ['Conclusion'],
            'future_work': ['Conclusion'],
            'conclusion': ['Conclusion'],
        }
        
        category = category.lower()
        
        # Get the list of columns corresponding to the specified category
        columns_to_search = category_mappings.get(category.lower())
        if not columns_to_search:
            raise ValueError(f"Invalid category: {category}")

        # Combine text from selected columns into a single string for ChatGPT input
        df['combined_text'] = df[columns_to_search].apply(lambda x: ' '.join(x.dropna()), axis=1)

        # Use ChatGPT to extract information based on the combined text
        def chatgpt_completion(text):
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                    "role": "system",
                    "content": f"Extract the {category} of the paper"
                },
                {
                    "role": "user",
                    "content": text
                }
                ]
            )
            return response.choices[0].message['content'].strip()

        df['ExtractedInfo'] = df['combined_text'].apply(chatgpt_completion)

        # Drop temporary combined_text column
        df.drop('combined_text', axis=1, inplace=True)

        return df
    
    except Exception as e:
        return f"Error: {str(e)}"



# Specify the category ('purpose', 'contribution', 'method', 'dataset', 'findings', 'future_work')
category = 'conclusion'

# Apply the function to extract information based on the specified category
result_df = extract_info_with_chatgpt(category, df)

# Display the updated DataFrame with extracted information
result_df


                                                Name  \
0  One Cannot Stand for Everyone! Leveraging Mult...   
1  SafeConv: Explaining and Correcting Conversati...   
2  Detecting and Mitigating Hallucinations in Mac...   
3  Explainable Recommendation with Personalized R...   
4     Binary and Ternary Natural Language Generation   
5  Span-Selective Linear Attention Transformers f...   
6  EMPre-training for Multi-party Dialogue Respon...   
7  ACLM: A Selective-Denoising based Generative D...   
8  Natural Language to Code Generation in Interac...   
9  Subset Retrieval Nearest Neighbor Machine Tran...   

                                            Abstract  \
0  Abstract User simulators are agents designed t...   
1  Abstract One of the main challenges opendomain...   
2  Abstract While the problem of hallucinations i...   
3  Abstract Explainable recommendation is a techn...   
4  Abstract Ternary and binary neural networks en...   
5  Abstract In schemaguided dialogue state trac

In [51]:
result_df

,Name,Abstract,Background,Introduction,Methodology,Experiments,Related Work,Conclusion,ExtractedInfo
0,One Cannot Stand for Everyone! Leveraging Mult...,Abstract User simulators are agents designed t...,Background Dialogue system. Taskoriented dialo...,Introduction Taskoriented dialogue systems aim...,method called MUST adaptive that balances i) t...,"experiments, we observed that the dialogue sys...",,"Conclusion In this paper, we propose a framewo...",The conclusion of the paper is that the propos...
1,SafeConv: Explaining and Correcting Conversati...,Abstract One of the main challenges opendomain...,,Introduction Safety of artificial intelligence...,method is onetime checking and rewriting—direc...,Experiments show that the detected unsafe beha...,Related Work Dialogue Safety Datasets Datasets...,"Conclusion In this paper, we study how to expl...",The conclusion of the paper highlights the dev...
2,Detecting and Mitigating Hallucinations in Mac...,Abstract While the problem of hallucinations i...,"Background and Setting In this section, we des...",Introduction Hallucinations in machine transla...,method that evaluates the percentage of the so...,experiments.1 1 Introduction Hallucinations in...,,Conclusions We start by asking how far we can...,The conclusion of the paper highlights the suc...
3,Explainable Recommendation with Personalized R...,Abstract Explainable recommendation is a techn...,,Introduction Recent years have witnessed a gro...,method has been employed to identify and selec...,experiments on three datasets show that our mo...,Related Work 2.1 Explainable Recommendation wi...,"Conclusion In this paper, we propose a novel m...","model, ERRA, outperforms existing methods in b..."
4,Binary and Ternary Natural Language Generation,Abstract Ternary and binary neural networks en...,,Introduction Generative pretrained transformer...,method to natural language generation tasks an...,"Experiments In this section, we evaluate the e...",Related Work Quantization has long been studie...,Conclusion We have demonstrated high accuracy ...,The paper concludes by highlighting the succes...
5,Span-Selective Linear Attention Transformers f...,Abstract In schemaguided dialogue state tracki...,,Introduction Dialogue State Tracking (DST) ref...,approach significantly improves upon existin...,experiments on the SchemaGuided Dialogue (SGD)...,Related Work Extractive DST. Following the tra...,"Conclusion In this work we introduced SPLAT, a...",The conclusion of the paper highlights the int...
6,EMPre-training for Multi-party Dialogue Respon...,Abstract Dialogue response generation requires...,,Introduction Inspired by the tremendous succes...,method. The official implementation of this pa...,experiments have justified the feasibility and...,Related Works 2.1 Pretraining for Response Ge...,Conclusion Most multiparty dialogue corpora ar...,The conclusion of the paper highlights the sig...
7,ACLM: A Selective-Denoising based Generative D...,Abstract Complex Named Entity Recognition (NER...,Background and Related Work Complex NER Backgr...,Introduction Named Entity Recognition (NER) is...,"method (Zhou and Chen, 2021) (previously evalu...",experiments reveal that the performance of the...,Related Work Complex NER,"Conclusion In this paper, we propose ACLM, a n...","In conclusion, the paper introduces ACLM, a no..."
8,Natural Language to Code Generation in Interac...,"Abstract Computational notebooks, such as Jupy...",background material and hints in tutorial note...,Introduction Data science is the process of ex...,Method Length All / pandas JuICe (Agashe et al...,Experiments Models We evaluate PACHINC Oand st...,Related Work Automating Data Science The amoun...,"Conclusion In this paper we present ARCADE , a...","The paper concludes by presenting ARCADE, a co..."
9,Subset Retrieval Nearest Neighbor Machine Tran...,Abstract knearest-neighbor machine translation...,,Introduction Neural machine translation (NMT) ...,method improves the trans

In [41]:
import pandas as pd

# Assuming df is your DataFrame containing the data you want to save

# Function to clean problematic characters
def clean_text(text):
    try:
        # Encode text to handle surrogate characters
        cleaned_text = text.encode('utf-8', errors='ignore').decode('utf-8')
        return cleaned_text
    except:
        return None

# Apply clean_text function to all columns of DataFrame
df_cleaned = df.applymap(clean_text)

# Export cleaned DataFrame to JSON
json_file_path = 'workout.json'
df_cleaned.to_json(json_file_path, orient='records', force_ascii=False)
print(f"Data saved to {json_file_path}")


Data saved to workout.json


C:\Users\harsh\AppData\Local\Temp\ipykernel_25564\2892372394.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(clean_text)


In [42]:
# Specify the file path for saving the JSON file locally
json_file_path = r'C:\Users\harsh\Downloads\workout.json'

# Export cleaned DataFrame to JSON with specified file path
df_cleaned.to_json(json_file_path, orient='records', force_ascii=False)

print(f"Data saved to {json_file_path}")

Data saved to C:\Users\harsh\Downloads\workout.json
